This is from a dataset I found on Kaggle about lung cancer and I am doing a prediction models on.

[https://www.kaggle.com/datasets/shantanugarg274/lung-cancer-prediction-dataset?resource=download]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, r2_score, mean_squared_error


In [9]:
# Load the dataset, print
df = pd.read_csv("Lung Cancer Dataset.csv")
print("Shape of dataset:", df.shape)
df.head(5)

Shape of dataset: (5000, 18)


,AGE,GENDER,SMOKING,FINGER_DISCOLORATION,MENTAL_STRESS,EXPOSURE_TO_POLLUTION,LONG_TERM_ILLNESS,ENERGY_LEVEL,IMMUNE_WEAKNESS,BREATHING_ISSUE,ALCOHOL_CONSUMPTION,THROAT_DISCOMFORT,OXYGEN_SATURATION,CHEST_TIGHTNESS,FAMILY_HISTORY,SMOKING_FAMILY_HISTORY,STRESS_IMMUNE,PULMONARY_DISEASE
0,68,1,1,1,1,1,0,57.831178,0,0,1,1,95.977287,1,0,0,0,NO
1,81,1,1,0,0,1,1,47.694835,1,1,0,1,97.184483,0,0,0,0,YES
2,58,1,1,0,0,0,0,59.577435,0,1,1,0,94.974939,0,0,0,0,NO
3,44,0,1,0,1,1,0,59.785767,0,1,0,1,95.187900,0,0,0,0,YES
4,72,0,1,1,1,1,1,59.733941,0,1,0,1,93.503008,0,0,0,0,YES


In [3]:
# Check for missing values
print(df.isnull().sum())

# Target variable distribution
print("\nPULMONARY_DISEASE value counts:")
print(df["PULMONARY_DISEASE"].value_counts())

AGE                       0
GENDER                    0
SMOKING                   0
FINGER_DISCOLORATION      0
MENTAL_STRESS             0
EXPOSURE_TO_POLLUTION     0
LONG_TERM_ILLNESS         0
ENERGY_LEVEL              0
IMMUNE_WEAKNESS           0
BREATHING_ISSUE           0
ALCOHOL_CONSUMPTION       0
THROAT_DISCOMFORT         0
OXYGEN_SATURATION         0
CHEST_TIGHTNESS           0
FAMILY_HISTORY            0
SMOKING_FAMILY_HISTORY    0
STRESS_IMMUNE             0
PULMONARY_DISEASE         0
dtype: int64

PULMONARY_DISEASE value counts:
PULMONARY_DISEASE
NO     2963
YES    2037
Name: count, dtype: int64


There is no nan values. The output indicates no missing values in any column. The target distribution shows about 2037 "YES" cases and 2963 "NO" cases, meaning approximately 40.7% of the patients have a pulmonary disease and 59.3% do not. Thus, the classes are somewhat imbalanced but not severely.

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
AGE,5000.0,57.222800,15.799224,30.000000,44.000000,57.000000,71.000000,84.000000
GENDER,5000.0,0.501200,0.500049,0.000000,0.000000,1.000000,1.000000,1.000000
SMOKING,5000.0,0.666400,0.471546,0.000000,0.000000,1.000000,1.000000,1.000000
FINGER_DISCOLORATION,5000.0,0.601200,0.489700,0.000000,0.000000,1.000000,1.000000,1.000000
MENTAL_STRESS,5000.0,0.539800,0.498463,0.000000,0.000000,1.000000,1.000000,1.000000
EXPOSURE_TO_POLLUTION,5000.0,0.516000,0.499794,0.000000,0.000000,1.000000,1.000000,1.000000
LONG_TERM_ILLNESS,5000.0,0.439200,0.496339,0.000000,0.000000,0.000000,1.000000,1.000000
ENERGY_LEVEL,5000.0,55.032043,7.913083,23.258308,49.440685,55.050421,60.323320,83.046971
IMMUNE_WEAKNESS,5000.0,0.394800,0.488857,0.000000,0.000000,0.000000,1.000000,1.000000
BREATHING_ISSUE,5000.0,0.800400,0.399740,0.000000,1.000000,1.000000,1.000000,1.000000


From the summary, the average age is about 57 years (min 30, max 84). About 50% of the patients are male (GENDER median 1.0) and 66.6% are smokers (SMOKING mean 0.667).

 We also observe a high prevalence of certain symptoms in this dataset: for example, 80% of individuals have breathing issues and 70% report throat discomfort. This suggests the dataset may largely consist of people with respiratory symptoms. However, not all symptomatic individuals have the disease, as only 40% are diagnosed with pulmonary disease.

In [5]:
# Compare pulmonary disease incidence among smokers vs non-smokers
smoker_ct = pd.crosstab(df["SMOKING"], df["PULMONARY_DISEASE"], normalize="index")
print(smoker_ct)

PULMONARY_DISEASE        NO       YES
SMOKING                              
0                  0.913070  0.086930
1                  0.432173  0.567827


The crosstab reveals a stark difference: among smokers, about 56.8% have pulmonary disease, whereas among non-smokers, only about 8.7% have the disease. This indicates smoking is a strong risk factor. Similarly, having a family history of smoking is associated with higher disease incidence (about 70% vs 33% if no family smoking history). In contrast, features like AGE and GENDER show little to no difference between the disease groups – the average age of patients with and without disease is almost the same (about 57 years), and both males and females have roughly equal disease rates (around 41%). This suggests that in this dataset, lifestyle factors and symptoms (like smoking, throat discomfort, breathing issues) are more indicative of pulmonary disease than basic demographics (age, sex).

In [6]:
df_enc = df.copy()
df_enc["PULMONARY_DISEASE_BINARY"] = df_enc["PULMONARY_DISEASE"].map({"YES": 1, "NO": 0})
corr_target = df_enc.corr(numeric_only=True)["PULMONARY_DISEASE_BINARY"].drop("PULMONARY_DISEASE_BINARY")
# Get top 5 features by absolute correlation
top5_corr = corr_target.abs().sort_values(ascending=False).head(5)
print("Top 5 correlated features with Pulmonary Disease:")
print(top5_corr)

Top 5 correlated features with Pulmonary Disease:
SMOKING                   0.461467
SMOKING_FAMILY_HISTORY    0.302478
THROAT_DISCOMFORT         0.283545
BREATHING_ISSUE           0.270464
STRESS_IMMUNE             0.181053
Name: PULMONARY_DISEASE_BINARY, dtype: float64


After encoding, we identify the top 5 features most correlated with the target (by absolute Pearson correlation).
They are:

SMOKING – Highest correlation (about 0.46) with pulmonary disease. (Smokers are far more likely to have the disease, as noted above.)
SMOKING_FAMILY_HISTORY – Correlation about 0.30. (Having a family history of smoking elevates risk, possibly due to second-hand smoke or genetic predisposition.)

THROAT_DISCOMFORT – Correlation about 0.28. (Persistent throat discomfort is a symptom strongly associated with the disease.)

BREATHING_ISSUE – Correlation about 0.27. (Breathing difficulties are common in those with lung ailments.)

STRESS_IMMUNE – Correlation about 0.18. (This feature indicates stress-related immune weakness, which appears moderately associated with disease.)

For completeness, the next highest factors were ENERGY_LEVEL (about 0.17) and IMMUNE_WEAKNESS (about 0.12), but their correlations are substantially lower. Notably, AGE and GENDER have near-zero correlation with the target (-0.006 and -0.004 respectively), confirming they are not predictive in this dataset. Also, symptoms like chest tightness and finger discoloration, despite being present in many patients, show very low direct correlation (about 0.02) with the outcome – likely because they are common in both sick and non-sick groups.


#Training time

Data Preprocessing

In [7]:
df["PULMONARY_DISEASE_BINARY"] = df["PULMONARY_DISEASE"].map({"YES": 1, "NO": 0})

# Define feature matrix X and target y
X = df.drop(columns=["PULMONARY_DISEASE", "PULMONARY_DISEASE_BINARY"])
y = df["PULMONARY_DISEASE_BINARY"]

# Split into train and test sets (80% train, 20% test), stratifying to maintain class balance
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Training set size:", X_train.shape[0], "Test set size:", X_test.shape[0])

Training set size: 4000 Test set size: 1000


We added a new column PULMONARY_DISEASE_BINARY as our numeric target (1 = YES, 0 = NO).

All predictor features are already numeric, so no additional encoding is needed. (If any were categorical strings, we would apply one-hot encoding or label encoding as appropriate, but here it’s not required.)

We perform an 80/20 train-test split, using stratification to ensure the ~40/60 class ratio is preserved in both sets. The training set has 4000 samples and the test set 1000 samples.

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Standardization. We will scale all features using StandardScaler, fitting on the training data and then transforming both train and test sets (to avoid any information leakage from the test set)

In [11]:
rf_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(kernel="rbf", probability=True, random_state=42)  # RBF kernel SVM
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Train the classifiers
rf_clf.fit(X_train_scaled, y_train)
svm_clf.fit(X_train_scaled, y_train)
xgb_clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:05:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [16]:
# Predictions and predicted probabilities on test set
y_pred_rf = rf_clf.predict(X_test_scaled)
y_pred_svm = svm_clf.predict(X_test_scaled)
y_pred_xgb = xgb_clf.predict(X_test_scaled)

# Predicted probabilities for positive class (needed for AUC)
proba_rf = rf_clf.predict_proba(X_test_scaled)[:, 1]
proba_svm = svm_clf.predict_proba(X_test_scaled)[:, 1]
proba_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]

In [17]:
rf_reg = RandomForestRegressor(random_state=42)
svr_reg = SVR(kernel="rbf")
xgb_reg = XGBRegressor(random_state=42)

# Train the regressors
rf_reg.fit(X_train_scaled, y_train)
svr_reg.fit(X_train_scaled, y_train)
xgb_reg.fit(X_train_scaled, y_train)

# Predictions on test set
y_pred_rf_reg = rf_reg.predict(X_test_scaled)
y_pred_svr_reg = svr_reg.predict(X_test_scaled)
y_pred_xgb_reg = xgb_reg.predict(X_test_scaled)


In [22]:
from sklearn.metrics import accuracy_score, roc_auc_score, r2_score, mean_squared_error

# Classification metrics
acc_rf = accuracy_score(y_test, y_pred_rf)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
auc_rf = roc_auc_score(y_test, proba_rf)
auc_svm = roc_auc_score(y_test, proba_svm)
auc_xgb = roc_auc_score(y_test, proba_xgb)

print("Classification Results on Test Set:")
print(f"Random Forest - Accuracy: {acc_rf:.3f}, AUC: {auc_rf:.3f}")
print(f"SVM           - Accuracy: {acc_svm:.3f}, AUC: {auc_svm:.3f}")
print(f"XGBoost       - Accuracy: {acc_xgb:.3f}, AUC: {auc_xgb:.3f}")

# Regression metrics
r2_rf = r2_score(y_test, y_pred_rf_reg)
r2_svr = r2_score(y_test, y_pred_svr_reg)
r2_xgb = r2_score(y_test, y_pred_xgb_reg)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf_reg))
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr_reg))
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb_reg))

print("\nRegression Results on Test Set:")
print(f"Random Forest - R²: {r2_rf:.3f}, RMSE: {rmse_rf:.3f}")
print(f"SVR           - R²: {r2_svr:.3f}, RMSE: {rmse_svr:.3f}")
print(f"XGBoost       - R²: {r2_xgb:.3f}, RMSE: {rmse_xgb:.3f}")

Classification Results on Test Set:
Random Forest - Accuracy: 0.904, AUC: 0.923
SVM           - Accuracy: 0.889, AUC: 0.924
XGBoost       - Accuracy: 0.879, AUC: 0.908

Regression Results on Test Set:
Random Forest - R²: 0.618, RMSE: 0.303
SVR           - R²: 0.608, RMSE: 0.308
XGBoost       - R²: 0.578, RMSE: 0.319


##Classification performance:

All three classifiers perform well, with high accuracy (~88–90%) and AUC around 0.91–0.92, indicating the models are able to distinguish between patients with and without pulmonary disease fairly reliably. Among them, the Random Forest classifier achieved the highest accuracy at 90.4%, while the SVM classifier achieved the highest AUC (approximately 0.924 vs. Random Forest’s 0.923, essentially a tie). The XGBoost classifier trails slightly with ~88.2% accuracy and 0.911 AUC. These results suggest that the models, especially Random Forest and SVM, are capturing the signal in the data (likely driven by features like smoking, symptoms, etc.) effectively. Regression performance: Treating the problem as regression yields significantly lower performance metrics in comparison.

The best regressor (Random Forest) obtained an R² of 0.618, meaning it explains about 61.8% of the variance in the 0/1 target, and an RMSE of ~0.303. The SVR is close behind (R² ~0.608), and XGBoost regressor is a bit lower (R² ~0.582). An R² around 0.60 is much lower (on a 0–1 scale) than the classification AUC ~0.92. This is expected because regression is not inherently well-suited to predicting a binary outcome – the models output continuous values that we effectively hope align to 0 or 1. In contrast, classification models are directly optimizing separation of classes.

#Conclusion
In summary, modeling the pulmonary disease outcome as a classification problem is far more effective than as a regression problem. The classification models achieved high accuracy (~90%) and AUC (about 0.92), whereas the regression models topped out around R² ~0.61. The nature of the target (YES/NO) is categorical, so a classification approach is the appropriate choice. Among the algorithms tested, the Random Forest classifier performed the best overall, attaining the highest accuracy (90.4%) while also nearly the highest AUC. The SVM classifier was a close second, slightly outperforming Random Forest in AUC by a negligible margin, but with a bit lower accuracy. The XGBoost model, while performing well, was marginally behind the other two in both metrics. For the regression approach, Random Forest Regressor was again the top performer (highest R² and lowest RMSE), but still notably inferior to its classification counterpart on this task. Conclusion: The classification approach using a Random Forest is the best choice for predicting pulmonary disease in this dataset. It provides the most accurate and robust predictions (AUC ~0.92), leveraging the strong signal of risk factors like smoking and related symptoms. In contrast, formulating the task as regression yields weaker performance and is not recommended for a binary outcome. The results underscore that using an appropriate modeling approach (classification for categorical targets) and a strong algorithm (Random Forest) leads to the best performance for pulmonary disease prediction in this data.